## Issues with the MedSAM2 Code base:



1.   Data must be loaded in a specfic format, by default it only has support for the BCTV Dataset in 3D (https://github.com/SuperMedIntel/Medical-SAM2/issues/75, https://github.com/SuperMedIntel/Medical-SAM2/issues/55)
2.   _C.so file issues and other issues(https://github.com/SuperMedIntel/Medical-SAM2/issues/26, https://github.com/SuperMedIntel/Medical-SAM2/issues/64)
3.    CUDA Version Incompatibilities


In [ ]:
!git clone https://github.com/facebookresearch/segment-anything-2

Cloning into 'segment-anything-2'...
remote: Enumerating objects: 974, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 974 (delta 17), reused 18 (delta 6), pack-reused 936 (from 1)
Receiving objects: 100% (974/974), 128.94 MiB | 15.96 MiB/s, done.
Resolving deltas: 100% (334/334), done.


In [ ]:
!git clone https://github.com/MedicineToken/Medical-SAM2.git

Cloning into 'Medical-SAM2'...
remote: Enumerating objects: 191, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 191 (delta 50), reused 42 (delta 42), pack-reused 124 (from 1)
Receiving objects: 100% (191/191), 2.20 MiB | 30.04 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [ ]:
%cd /content/segment-anything-2
!pip install -q -e .

/content/segment-anything-2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.8 MB/s eta 0:00:00
  Building editable for SAM-2 (pyproject.toml) ... done


In [ ]:
import os
import pandas as pd
import cv2
import torch
import torch.nn.utils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.model_selection import train_test_split
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

In [ ]:
%cd ..

/content


In [ ]:
%%writefile /content/Medical-SAM2/setup.py

# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
import os

from setuptools import find_packages, setup

# Package metadata
NAME = "SAM 2"
VERSION = "1.0"
DESCRIPTION = "SAM 2: Segment Anything in Images and Videos"
URL = "https://github.com/facebookresearch/segment-anything-2"
AUTHOR = "Meta AI"
AUTHOR_EMAIL = "segment-anything@meta.com"
LICENSE = "Apache 2.0"

# Read the contents of README file
with open("README.md", "r", encoding="utf-8") as f:
    LONG_DESCRIPTION = f.read()

# Required dependencies
REQUIRED_PACKAGES = [
    "torch>=2.3.1",
    "torchvision>=0.18.1",
    "numpy>=1.24.4",
    "tqdm>=4.66.1",
    "hydra-core>=1.3.2",
    "iopath>=0.1.10",
    "pillow>=9.4.0",
]

EXTRA_PACKAGES = {
    "demo": ["matplotlib>=3.9.1", "jupyter>=1.0.0", "opencv-python>=4.7.0"],
    "dev": ["black==24.2.0", "usort==1.0.2", "ufmt==2.0.0b2"],
}

# By default, we also build the SAM 2 CUDA extension.
# You may turn off CUDA build with `export SAM2_BUILD_CUDA=0`.
BUILD_CUDA = os.getenv("SAM2_BUILD_CUDA", "1") == "1"
# By default, we allow SAM 2 installation to proceed even with build errors.
# You may force stopping on errors with `export SAM2_BUILD_ALLOW_ERRORS=0`.
BUILD_ALLOW_ERRORS = os.getenv("SAM2_BUILD_ALLOW_ERRORS", "1") == "1"

# Catch and skip errors during extension building and print a warning message
# (note that this message only shows up under verbose build mode
# "pip install -v -e ." or "python setup.py build_ext -v")
CUDA_ERROR_MSG = (
    "{}\n\n"
    "Failed to build the SAM 2 CUDA extension due to the error above. "
    "You can still use SAM 2 and it's OK to ignore the error above, although some "
    "post-processing functionality may be limited (which doesn't affect the results in most cases; "
    "(see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).\n"
)


def get_extensions():
    if not BUILD_CUDA:
        return []

    try:
        from torch.utils.cpp_extension import CUDAExtension

        srcs = ["sam2_train/csrc/connected_components.cu"]
        compile_args = {
            "cxx": [],
            "nvcc": [
                "-DCUDA_HAS_FP16=1",
                "-D__CUDA_NO_HALF_OPERATORS__",
                "-D__CUDA_NO_HALF_CONVERSIONS__",
                "-D__CUDA_NO_HALF2_OPERATORS__",
            ],
        }
        ext_modules = [CUDAExtension("sam2_train._C", srcs, extra_compile_args=compile_args)]
    except Exception as e:
        if BUILD_ALLOW_ERRORS:
            print(CUDA_ERROR_MSG.format(e))
            ext_modules = []
        else:
            raise e

    return ext_modules


try:
    from torch.utils.cpp_extension import BuildExtension

    class BuildExtensionIgnoreErrors(BuildExtension):

        def finalize_options(self):
            try:
                super().finalize_options()
            except Exception as e:
                print(CUDA_ERROR_MSG.format(e))
                self.extensions = []

        def build_extensions(self):
            try:
                super().build_extensions()
            except Exception as e:
                print(CUDA_ERROR_MSG.format(e))
                self.extensions = []

        def get_ext_filename(self, ext_name):
            try:
                return super().get_ext_filename(ext_name)
            except Exception as e:
                print(CUDA_ERROR_MSG.format(e))
                self.extensions = []
                return "_C.so"

    cmdclass = {
        "build_ext": (
            BuildExtensionIgnoreErrors.with_options(no_python_abi_suffix=True)
            if BUILD_ALLOW_ERRORS
            else BuildExtension.with_options(no_python_abi_suffix=True)
        )
    }
except Exception as e:
    cmdclass = {}
    if BUILD_ALLOW_ERRORS:
        print(CUDA_ERROR_MSG.format(e))
    else:
        raise e


# Setup configuration
setup(
    name=NAME,
    version=VERSION,
    description=DESCRIPTION,
    long_description=LONG_DESCRIPTION,
    long_description_content_type="text/markdown",
    url=URL,
    author=AUTHOR,
    author_email=AUTHOR_EMAIL,
    license=LICENSE,
    packages=find_packages(exclude="notebooks"),
    package_data={"": ["*.yaml"]},  # SAM 2 configuration files
    include_package_data=True,
    install_requires=REQUIRED_PACKAGES,
    extras_require=EXTRA_PACKAGES,
    python_requires=">=3.10.0",
    ext_modules=get_extensions(),
    cmdclass=cmdclass,
)

Writing /content/Medical-SAM2/setup.py


In [ ]:
# move to repo dir
%cd /content/Medical-SAM2

# build setup
!python setup.py build_ext --inplace

/content/Medical-SAM2
running build_ext
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:497: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:416: UserWarning: The detected CUDA version (12.2) has a minor version mismatch with the version that was used to compile PyTorch (12.1). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:426: UserWarning: There are no x86_64-linux-gnu-g++ version bounds defined for CUDA version 12.2
  warnings.warn(f'There are no {compiler_name} version bounds defined for CUDA version {cuda_str_version}')
building 'sam2_train._C' extension
creating build/temp.linux-x86_64-cpython-310/s

In [ ]:
!bash checkpoints/download_ckpts.sh

--2024-11-19 06:37:19--  https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.124, 108.157.254.121, 108.157.254.102, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155906050 (149M) [application/vnd.snesdev-page-table]
Saving to: ‘sam2_hiera_tiny.pt’

sam2_hiera_tiny.pt  100%[===================>] 148.68M   260MB/s    in 0.6s    

2024-11-19 06:37:19 (260 MB/s) - ‘sam2_hiera_tiny.pt’ saved [155906050/155906050]

--2024-11-19 06:37:19--  https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_small.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.124, 108.157.254.121, 108.157.254.102, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184309650 

In [ ]:
!mv /content/Medical-SAM2/sam2_hiera_base_plus.pt /content/Medical-SAM2/sam2_hiera_large.pt /content/Medical-SAM2/sam2_hiera_small.pt /content/Medical-SAM2/sam2_hiera_tiny.pt ./checkpoints/


In [ ]:
%%writefile /content/Medical-SAM2/conf/global_settings.py

import os
from datetime import datetime

CHECKPOINT_PATH = 'checkpoint'

#total training epoches
EPOCH = 100
step_size = 10
i = 1
MILESTONES = []
while i * 5 <= EPOCH:
    MILESTONES.append(i* step_size)
    i += 1

#initial learning rate
#INIT_LR = 0.1

#time of we run the script
TIME_NOW = datetime.now().strftime("%F_%H-%M-%S.%f")

#tensorboard log dir
LOG_DIR = 'runs'

#save weights file per SAVE_EPOCH epoch
SAVE_EPOCH = 10

Overwriting /content/Medical-SAM2/conf/global_settings.py


In [ ]:
import os
import pandas as pd
import cv2
import torch
import torch.nn.utils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.model_selection import train_test_split
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

In [ ]:
! pip install monai tensorboardx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.3 MB/s eta 0:00:00


In [ ]:
!unzip /content/drive/MyDrive/ACDC_Dataset/PreprocessedACDCSeq.zip

Streaming output truncated to the last 5000 lines.
  inflating: PreprocessedACDC copy/Test/image/patient125/30.jpg  
  inflating: PreprocessedACDC copy/Test/image/patient125/31.jpg  
  inflating: PreprocessedACDC copy/Test/image/patient125/25.jpg  
  inflating: PreprocessedACDC copy/Test/image/patient125/19.jpg  
  inflating: PreprocessedACDC copy/Test/image/patient125/4.jpg  
  inflating: PreprocessedACDC copy/Test/image/patient125/5.jpg  
  inflating: PreprocessedACDC copy/Test/image/patient125/7.jpg  
  inflating: PreprocessedACDC copy/Test/image/patient125/6.jpg  
  inflating: PreprocessedACDC copy/Test/image/patient125/2.jpg  
  inflating: PreprocessedACDC copy/Test/image/patient125/3.jpg  
  inflating: PreprocessedACDC copy/Test/image/patient125/1.jpg  
  inflating: PreprocessedACDC copy/Test/image/patient125/0.jpg  
  inflating: PreprocessedACDC copy/Test/image/patient122/8.jpg  
  inflating: PreprocessedACDC copy/Test/image/patient122/9.jpg  
  inflating: PreprocessedACDC copy/

### The line below is the training code, I have added preprocessing code to convert the ACDC dataset structure to match the one that MedSAM2 requires

In [ ]:
!python train_3d.py -net sam2 -exp_name BTCV_MedSAM2 -sam_ckpt ./checkpoints/sam2_hiera_tiny.pt -sam_config sam2_hiera_t -image_size 1024 -val_freq 1 -prompt bbox -prompt_freq 2 -dataset btcv -data_path /content/drive/MyDrive/NTU/Sorted_Processed_ACDC/Right_Ventricle -b 1
